In [1]:
import numpy as np
import pandas as pd

In [47]:
def get_rmse(model):
    pred = model.predict(X_test_combined)
    mse = mean_squared_error(test_target, pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, ' RMSE: ', np.round(rmse,3))
    return rmse

In [24]:
def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        # rmses.append(rmse)
    # return rmses

In [25]:
# Preprocessing for turn over rate
def tor_preprocessing(tor):
  tor_df = tor
  tor_val = tor_df['turn_over_rate']

  for i, val in enumerate(tor_val):
    if '정보' in val:
      tor_df.loc[i,'turn_over_rate'] = np.NAN
    elif val == '(9999%)' :
      tor_df.loc[i,'turn_over_rate'] = np.NAN
    elif '(' in val:
      tmp = float(val[1:-1].replace('%',''))*0.01
      tor_df.loc[i,'turn_over_rate'] = tmp

  return tor_df

In [26]:
# Preprocessing for financial variable
def fv_preprocessing(fv):
  fv_df = fv
  afv = fv_df['average_salary']
  tfv = fv_df['total_sale']

  for i, val in enumerate(afv):
    if type(val) == str:
        if '만원' in val:
          tmp = int(val[:-2].replace(',',''))
          fv_df.loc[i,'average_salary'] = tmp
        if '회사' in val:
          fv_df.loc[i,'average_salary'] = np.NAN
        if '수집' in val:
          fv_df.loc[i,'average_salary'] = np.NAN

  for i, val in enumerate(tfv):
    if type(val) == str:
        if '조원' in val:
          tmp = float(val[:-2].replace(',',''))*1000
          fv_df.loc[i,'total_sale'] = tmp
        if '억원' in val:
          tmp = float(val[:-2].replace(',',''))
          fv_df.loc[i,'total_sale'] = tmp
        if '억' in val:
          tmp = float(val[:-1].replace(',',''))
          fv_df.loc[i,'total_sale'] = tmp
          print(fv_df.loc[i,'total_sale'])
          print(i)
        if '회사' in val:
          fv_df.loc[i,'total_sale'] = np.NAN

  return fv_df

In [27]:
combined_data = pd.read_csv("/content/drive/MyDrive/bow_df.csv")


In [28]:

from sklearn.model_selection import train_test_split
combined_data = combined_data[combined_data['turn_over_rate']<1]
error_firm = ['동원홈푸드','휠라홀딩스','트리','와디즈','키위컴퍼니','줌인터넷','시선인터내셔널','브이티코스메틱','유니슨이테크','씨엠비대전방송','서울비젼','더메인즈','조은시스템']
for i in error_firm:
    combined_data = combined_data[combined_data.company_name != i]
combined_data.dropna(axis=0, inplace=True)
combined_data = combined_data.drop(['Unnamed: 0'], axis=1)
combined_data = fv_preprocessing(combined_data)
tor = combined_data['turn_over_rate']
combined_data.drop('turn_over_rate', axis=1, inplace=True)


In [29]:
train_input, test_input, train_target, test_target = train_test_split(combined_data, tor, test_size=0.2, random_state=42)

In [30]:
train_input

,company_name,adv,dadv,average_salary,total_sale
701,나이스평가정보,"야근식대 제공, 자유로운 연차사용, 시차출퇴근제 가능, 복지 좋음 동절기 1시간 일...","사람을 부품처럼 사용, 인력부족이나 인턴채용으로만 메꾸려고 하여 남은 사람을 업무 ...",9691,4229.7
2372,삼성전자서비스,복지가 좋다 보너스 잘 나오고 연차가 자유롭고 직원과 친하게 지낼수 있다는 장점이 ...,고객에게 호구잡히고 빌고 기고 바뀌지 않음 고정급여가 아니라 죽어라 일을 해야함 실...,6076,1700.0
1419,안다르,E사 들어오고 매출은 올랐음사옥이 새거라서 깔끔함야근강요 같은건 없음 역세권이며 주...,브랜드가 너무 쌈마이 느낌임. 패션브랜드에게는 썩 좋은건 아님.광고멘트도 낯간지럽고...,4039,1144.2
392,체리쉬,근무강도가 세지 않고 연령대가 낮아 복장이나 규율은 다소 프리함.. 부서별로 다르지...,한 사람의 영향력이 지나치게 쎄다보니 조직개편과 내쫓기듯 나가는 사람들… 능력보다 ...,4750,157.6
1058,아모레퍼시픽,자사 제품 복지몰 구매 가능(50%할인 및 100만원 상당의 포인트)다양한 업무 경...,급하게 변하는 결정들로 인해갑자기 변경되는 계획들모든 크고 작은 결정들을하나부터 열...,6999,3500.0
...,...,...,...,...,...
1766,엠에스오토텍,아이크림을 준다 통큰버스가 다닌다 돈을 많이 받을 수 있다 일이 어렵지 않다 밥을 ...,계약직만 뽑는다 시도때도 없는 현장 지원 책임전가 70~80년대 마인드 들에 꼰대들...,6830,1806.8
1182,하이에어코리아,식권 나와서 점심은 식당 4곳에서 골라먹을 수 있음사상역 근처라 교통이 편리함 부산...,"모든 것, 근무시간에는 전화 받느냐고 업무 못하고 퇴근시간 되서야 내 업무를 할 ...",5046,2544.7
1220,한국남부발전,워라밸 - 특히 유연근무가 매우 잘되어 있음적당한 연봉 - 어디가서 굶어죽지는 않을...,"근무지가 정말 오지 중의 오지에 있다 삼척, 하동 등등.. 나머지는 그래도 살만하지...",8665,5700.0
1398,그린광학,좋은 사람이 많았다.구내식당이 있어서 밥값 아낄 수 있었다.사내 카페가 싸다. 연차...,직무에 맞지않는 업무연봉이 매우매우매우 낮음팀끼리 화합해도 모자랄판에 서로 기싸움과...,4719,323.1


In [31]:
test_input

,company_name,adv,dadv,average_salary,total_sale
1768,동원금속,많은 업무를 경험할 수 있고 생각지도 못했던 업무에 대해서도 견문을 넓힐 수 있는 ...,잡플래닛에 있는 연봉이 실제 연봉. 뭐 성과금 나오면 얼마 되겠지 하는 행복회로는 ...,5933,2063.4
549,정상제이엘에스,"업계 대비 높은 연봉, 수업 스케줄이 유연한 편이라 오후 출근 가능 업무자율성이 높...",아무래도 아이들과 학부모님을 상대해야하는 직업이기 때문에 힘이 들 수 밖에 없고 저...,4247,1011.3
1536,패커드코리아,휴가 자유지방 기업 대비 초봉 높은편자동차계열로 복지도 좋은편 점심밥이 제공된다.연...,지역이 조금 아쉬움.. 다만 차량만 있으면 위치가 좋음천안 번화가에서 차로 20분거...,6032,2201.5
44,현대하이라이프손해사정,"1. 자회사 탑(급여, 복지)2. 현대라는 타이틀3. 구내식당4. 훌륭한 전산5. ...",1. 보수적인 기업 문화2. 구린 사원증3. 업무환경 별로4. 화장실 별로 업무량 ...,5601,719.0
1509,일진다이아몬드,"연차를 생각보다 자유롭게 쓸 수 있으며, 식사도 생각보다 괜찮게 나오는 듯 연차를...",진천군에 있어서 거리가 출퇴근하기에는 조금 멀다. 자차가 없는 사람은 사내 기숙사 ...,4750,616.1
...,...,...,...,...,...
1968,지엔앰퍼포먼스,체계가 잡혀있는 편이라 좋긴 한데 그게 오히려 독일지도.. 체계와 절차가 너무 많다...,유연하지 못한 사고. 들어오는 일만 하는 느낌. 같이 있으면 나도 고이겠다 싶었음....,4675,188.5
1808,갑진,연차를 2일전까지만 말하면 되고 그리고 기숙사 시설이 좋습니다 업무체계 및 제한사항...,연봉이 낮고 꼰대들이 너무 많아요 그리고 중소기업이라 그런가 현장직처럼 일하는 경우...,4585,480.6
1956,채널에이,사람을 함부로 자르지는 않아 매우 안정적임. 구내 식당. 지방에 갈 걱정이 없음 광...,고인물들이 많고 변화를 좋아하지 않음. 소수의 경영진이 거의 모든 의사 결정을 내림...,5736,2066.9
2132,하나제약,연차사용이 비교적 자유롭다밥 맛있다초봉은 높다장점이 근데 너무 없다 이거빼고 다 단...,회사는 매년 성장하지만 직원 대우를 안해준다연차 쌓일수록 연봉성장률 없음올해 연봉동...,6220,1964.1


In [32]:
train_input = fv_preprocessing(train_input)

In [33]:
train_adv_text = train_input['adv'].to_numpy()
train_dadv_text = train_input['dadv'].to_numpy()

In [34]:
test_adv_text = test_input['adv'].to_numpy()
test_dadv_text = test_input['dadv'].to_numpy()

In [35]:
financial_train1=train_input['average_salary'].to_numpy()
financial_train2=train_input['total_sale'].to_numpy()
financial_test1=test_input['average_salary'].to_numpy()
financial_test2=test_input['total_sale'].to_numpy()

In [36]:
# np.array2string(train_adv_text)
# np.array2string(train_dadv_text)
# np.array2string(test_adv_text)
# np.array2string(test_dadv_text)

In [37]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

adv_tfidfvector = TfidfVectorizer()
adv_train = adv_tfidfvector.fit_transform(train_adv_text)
adv_test = adv_tfidfvector.transform(test_adv_text)

In [38]:
dadv_tfidfvector = TfidfVectorizer()
dadv_train = dadv_tfidfvector.fit_transform(train_dadv_text)
dadv_test = dadv_tfidfvector.transform(test_dadv_text)

In [39]:
financial_train1 = financial_train1.reshape(-1, 1)
financial_train2 = financial_train2.reshape(-1, 1)
financial_test1 = financial_test1.reshape(-1, 1)
financial_test2 = financial_test2.reshape(-1, 1)

In [40]:
financial_train1 = financial_train1.astype(float)
financial_train2 = financial_train2.astype(float)
financial_test1 = financial_test1.astype(float)
financial_test2 = financial_test2.astype(float)

In [41]:
from scipy.sparse import hstack # tf-idf로 변환된 텍스트 데이터와 금융 데이터를 연결
X_train_combined = hstack([adv_train, dadv_train, financial_train1, financial_train2])  # 텍스트 데이터와 금융 데이터를 수평으로 연결
X_test_combined = hstack([adv_test, dadv_test,financial_test1, financial_test2])    # 테스트 데이터에 대해서도 동일하게 수행

In [42]:
train_target = train_target.to_numpy()
test_target = test_target.to_numpy()

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score





lr_reg = LinearRegression()
lr_reg.fit(X_train_combined, train_target)

ridge_reg = Ridge()
ridge_reg.fit(X_train_combined, train_target)

lasso_reg = Lasso()
lasso_reg.fit(X_train_combined, train_target)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)
print(lr_reg.score(X_train_combined, train_target), ridge_reg.score(X_train_combined, train_target), lasso_reg.score(X_train_combined, train_target))
print(lr_reg.score(X_test_combined, test_target), ridge_reg.score(X_test_combined, test_target), lasso_reg.score(X_test_combined, test_target))


LinearRegression  RMSE:  0.162
Ridge  RMSE:  0.165
Lasso  RMSE:  0.187
0.965922044770044 0.9163368872321918 0.19748643462972826
0.3998918280791518 0.3781682737676828 0.2033037376798369


In [22]:
from lightgbm import LGBMRegressor

lgbm_params = {'n_estimators': [1000]}
lgbm_reg = LGBMRegressor(n_estimators=500, learning_rate= 0.1, num_leaves=4,
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
lgbm_reg.fit(X_train_combined, train_target)
print(lr_reg.score(X_train_combined, train_target))
print(lgbm_reg.score(X_test_combined, test_target))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.464948 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 388719
[LightGBM] [Info] Number of data points in the train set: 1844, number of used features: 13016
[LightGBM] [Info] Start training from score 0.314235
0.965922044770044


/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1019: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.3782909963100214


In [ ]:
X_train_combined.shape

(99, 38268)

In [ ]:
adv_train.shape

(99, 15157)

In [ ]:
dadv_train.shape

(99, 23109)